# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
import re
import nltk
nltk.download(['punkt', 'wordnet'])
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report
import pickle

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [2]:
# load data from database
engine = create_engine('sqlite:////home/workspace/disasterResponse.db')
df = pd.read_sql('disasterResponse', engine)

df.head()
X = df['message']
y = df.drop(['id', 'message', 'original', 'genre'], axis=1)

### 2. Write a tokenization function to process your text data

In [3]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

def tokenize(text):
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, 'urlplaceholder')
    
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
        
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X,y)
#train classifier
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [6]:
#predict on the test data
y_pred = pipeline.predict(X_test)

#classification report
def each_col_report(y_test, y_pred):
    for i, col in enumerate(y_test.columns): 
        print('---------------------------------')
        print(col)
        print(classification_report(y_test.iloc[:,i], y_pred[:,i]))

each_col_report(y_test, y_pred)

---------------------------------
related
             precision    recall  f1-score   support

          0       0.61      0.36      0.45      1466
          1       0.83      0.93      0.88      5037
          2       0.80      0.16      0.26        51

avg / total       0.78      0.80      0.78      6554

---------------------------------
request
             precision    recall  f1-score   support

          0       0.88      0.98      0.93      5419
          1       0.80      0.35      0.48      1135

avg / total       0.86      0.87      0.85      6554

---------------------------------
offer
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      6520
          1       0.00      0.00      0.00        34

avg / total       0.99      0.99      0.99      6554

---------------------------------
aid_related
             precision    recall  f1-score   support

          0       0.72      0.88      0.79      3836
          1       0.75   

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [7]:
parameters = {
    'tfidf__norm':['l2','l1'],
    'clf__estimator__min_samples_split':[2,3],
    }

cv = GridSearchCV(pipeline, parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [8]:
cv.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'tfidf__norm': ['l2', 'l1'], 'clf__estimator__min_samples_split': [2, 3]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [9]:
y_pred = cv.predict(X_test)

In [10]:
accuracy = (y_pred == y_test).sum()/y_test.shape[0]
print('accuracy :', accuracy)
each_col_report(y_test, y_pred)

accuracy : related                   0.802716
request                   0.879616
offer                     0.994812
aid_related               0.719713
medical_help              0.923863
medical_products          0.951633
search_and_rescue         0.972688
security                  0.981385
military                  0.964602
child_alone               1.000000
water                     0.953769
food                      0.925847
shelter                   0.923711
clothing                  0.986115
money                     0.976503
missing_people            0.987183
refugees                  0.965822
death                     0.955294
other_aid                 0.872444
infrastructure_related    0.929204
transport                 0.954074
buildings                 0.950870
electricity               0.980317
tools                     0.993592
hospitals                 0.987641
shops                     0.995117
aid_centers               0.987794
other_infrastructure      0.952243
weather_r

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [11]:
improved_pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

        ])),

        ('clf', MultiOutputClassifier(AdaBoostClassifier(n_estimators = 100)))
    ])


improved_pipeline.fit(X_train, y_train)
y_pred_improved = improved_pipeline.predict(X_test)
each_col_report(y_test, y_pred_improved)

---------------------------------
related
             precision    recall  f1-score   support

          0       0.55      0.28      0.37      1466
          1       0.81      0.93      0.87      5037
          2       0.38      0.20      0.26        51

avg / total       0.75      0.78      0.75      6554

---------------------------------
request
             precision    recall  f1-score   support

          0       0.91      0.96      0.94      5419
          1       0.77      0.55      0.64      1135

avg / total       0.89      0.89      0.89      6554

---------------------------------
offer
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      6520
          1       0.00      0.00      0.00        34

avg / total       0.99      0.99      0.99      6554

---------------------------------
aid_related
             precision    recall  f1-score   support

          0       0.77      0.86      0.81      3836
          1       0.76   

### 9. Export your model as a pickle file

In [12]:
# save the model to disk
pickle.dump(improved_pipeline, open('model.sav', 'wb'))
 
# some time later...
 
# load the model from disk
loaded_model = pickle.load(open('model.sav', 'rb'))
result = loaded_model.score(X_test, y_test)
print(result)

0.240616417455


In [13]:
# save the model to disk
pickle.dump(pipeline, open('first_model.sav', 'wb'))
 
# some time later...
 
# load the model from disk
loaded_model = pickle.load(open('model.sav', 'rb'))
result = loaded_model.score(X_test, y_test)
print(result)

0.240616417455


### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.